# Summary

Wrangle the all card data.  Joins set data as well.  Then selects Standard Legal cards and other 
interesting data sets.

In [1]:
# Setup Notebook
import os
if os.path.basename(os.getcwd()) != "mtg-modeling":
    get_ipython().run_line_magic("run", '-i "../../scripts/notebook_header.py"')  # type: ignore

Changed working directory to: /root/mtg-modeling


In [2]:
import pandas as pd
import polars as pl

In [3]:
paths = {
    "raw": Path("data/raw/mtgjson/AllPrintingsParquetFiles"),
    "interim": Path("data/interim/mtgjson/AllPrintings"),
    "processed": Path("data/processed/mtgjson/AllPrintings"),
}

paths['raw'].exists()
os.makedirs(paths["interim"], exist_ok=True)
os.makedirs(paths["processed"], exist_ok=True)

In [4]:
cards = pl.scan_parquet(paths["raw"] / "cards.parquet")
legalities = pl.scan_parquet(paths["raw"] / "cardLegalities.parquet")
purchase_urls = pl.scan_parquet(paths["raw"] / "cardPurchaseUrls.parquet")
sets = pl.scan_parquet(paths["raw"] / "sets.parquet")

In [5]:
cards.collect().columns

['artist',
 'artistIds',
 'asciiName',
 'attractionLights',
 'availability',
 'boosterTypes',
 'borderColor',
 'cardParts',
 'colorIdentity',
 'colorIndicator',
 'colors',
 'defense',
 'duelDeck',
 'edhrecRank',
 'edhrecSaltiness',
 'faceConvertedManaCost',
 'faceFlavorName',
 'faceManaValue',
 'faceName',
 'finishes',
 'flavorName',
 'flavorText',
 'frameEffects',
 'frameVersion',
 'hand',
 'hasAlternativeDeckLimit',
 'hasContentWarning',
 'hasFoil',
 'hasNonFoil',
 'isAlternative',
 'isFullArt',
 'isFunny',
 'isOnlineOnly',
 'isOversized',
 'isPromo',
 'isRebalanced',
 'isReprint',
 'isReserved',
 'isStarter',
 'isStorySpotlight',
 'isTextless',
 'isTimeshifted',
 'keywords',
 'language',
 'layout',
 'leadershipSkills',
 'life',
 'loyalty',
 'manaCost',
 'manaValue',
 'name',
 'number',
 'originalPrintings',
 'originalReleaseDate',
 'originalText',
 'originalType',
 'otherFaceIds',
 'power',
 'printings',
 'promoTypes',
 'rarity',
 'rebalancedPrintings',
 'relatedCards',
 'securitySt

In [6]:
results = (
    cards
    .join(sets, left_on='setCode', right_on='code', how='left')
    .join(legalities, on='uuid', how='left')
    .join(purchase_urls, on='uuid', how='left')
)

wide_cards = results.collect()
wide_cards.write_parquet(paths["interim"] / "wide_cards.parquet")
print(wide_cards.shape)

(96296, 127)


## Data for Standard Legal Cards

Used for analysis of the Standard format.  

See the [MTG Wiki Standard/Timeline](https://mtg.fandom.com/wiki/Standard/Timeline) page for validate correct card composition.

In [7]:
standard_results = (
    results
    .filter(pl.col('standard') == 'Legal')
    .filter(pl.col('borderColor') == 'black')
    .filter(pl.col('isPromo').is_null())
    # .filter(pl.col('isReprint').is_null())
    .filter(pl.col('promoTypes').is_null())
    .sort(pl.col('name'), descending=False)
    .sort(pl.col('releaseDate'), descending=False)
    .sort(pl.col('power'), descending=False)
)

standard_cards = standard_results.collect()
standard_cards.write_parquet(paths["processed"] / "standard_cards.parquet")
print(standard_cards.shape)

(6349, 127)


## Outlaws of Thunder Junction Standard Legal Set

Per the [wiki](https://mtg.fandom.com/wiki/Standard/Timeline), there should be 271 cards in the OTJ set.

In [12]:
otj_results = (
    results
    .filter(pl.col('standard') == 'Legal')
    .filter(pl.col('borderColor') == 'black')
    .filter(pl.col('isPromo').is_null())
    # .filter(pl.col('isReprint').is_null())
    .filter(pl.col('promoTypes').is_null())
    .filter(pl.col('setCode') == 'OTJ')
    .sort(pl.col('number'))
)

otj_cards = otj_results.collect()
otj_cards.write_parquet(paths["processed"] / "OTJ_std_cards.parquet")
otj_cards_ids = (
    otj_cards.select([pl.col("name"), pl.col("number").str.zfill(3)])
    .sort(pl.col("number"))
    .to_pandas()
)

In [13]:
otj_core_results = (
    otj_results
    .select([
        pl.col('name'),
        pl.col('setCode'),
        pl.col('releaseDate'),
        pl.col('number').str.zfill(3),
        pl.col('layout'),
        pl.col('availability'),
        pl.col('power'),
        pl.col('toughness'),
        pl.col('colorIdentity'),
        pl.col('colors'),
        pl.col('types'),
        pl.col('subtypes'),
        pl.col('supertypes'),
        pl.col('manaCost'),
        pl.col('manaValue'),
        pl.col('edhrecRank'),
        pl.col('edhrecSaltiness'),
        pl.col('text'),
        pl.col('flavorText'),
    ])
)

otj_core_cards = otj_core_results.collect()
otj_core_cards.write_parquet(paths["processed"] / "OTJ_std_thin.parquet")
otj_core_cards.head()

name,setCode,releaseDate,number,layout,availability,power,toughness,colorIdentity,colors,types,subtypes,supertypes,manaCost,manaValue,edhrecRank,edhrecSaltiness,text,flavorText
str,str,str,str,str,str,str,str,str,str,str,str,str,str,f64,f64,f64,str,str
"""Another Round""","""OTJ""","""2024-04-19""","""001""","""normal""","""arena, mtgo, paper""",null,null,"""W""","""W""","""Sorcery""","""""","""""","""{X}{X}{2}{W}""",3.0,8442.0,null,"""Exile any number of creatures …","""Weary travelers trade stories …"
"""Eriette's Lullaby""","""OTJ""","""2024-04-19""","""010""","""normal""","""arena, mtgo, paper""",null,null,"""W""","""W""","""Sorcery""","""""","""""","""{1}{W}""",2.0,15133.0,null,"""Destroy target tapped creature…","""When Kellan questioned her, Er…"
"""Rattleback Apothecary""","""OTJ""","""2024-04-19""","""100""","""normal""","""arena, mtgo, paper""","""3""","""2""","""B""","""B""","""Creature""","""Gorgon, Warlock""","""""","""{2}{B}""",3.0,13674.0,null,"""Deathtouch\nWhenever you commi…","""""Looking for a little liquid c…"
"""Raven of Fell Omens""","""OTJ""","""2024-04-19""","""101""","""normal""","""arena, mtgo, paper""","""1""","""2""","""B""","""B""","""Creature""","""Bird""","""""","""{1}{B}""",2.0,13160.0,null,"""Flying\nWhenever you commit a …","""A raven sighted at midnight of…"
"""Rictus Robber""","""OTJ""","""2024-04-19""","""102""","""normal""","""arena, mtgo, paper""","""4""","""3""","""B""","""B""","""Creature""","""Zombie, Rogue""","""""","""{3}{B}""",4.0,19562.0,null,"""When Rictus Robber enters, if …",null
